In [ ]:
import datetime
import json
import markdown2  # Install it using: pip install markdown2

class Category:
    def __init__(self, name):
        self.name = name
        self.notes = []

    def add_note_to_category(self, note):
        self.notes.append(note)

    def view_notes_in_category(self):
        if not self.notes:
            print(f"No notes in the '{self.name}' category.")
        else:
            print(f"Notes in the '{self.name}' category:")
            for i, note in enumerate(self.notes, 1):
                print(f"Note {i}:\n{note.render_content()}")

class Note:
    def __init__(self, title, content, categories=None, tags=None):
        self.title = title
        self.content = content
        self.created_at = datetime.datetime.now()
        self.updated_at = self.created_at
        self.categories = categories if categories else []
        self.tags = tags if tags else []
        self.is_task = False
        self.priority = None
        self.completed_at = None

    def update_content(self, new_content):
        self.content = new_content
        self.updated_at = datetime.datetime.now()

    def add_category(self, category):
        self.categories.append(category)

    def add_tag(self, tag):
        self.tags.append(tag)

    def mark_as_task(self, priority=None):
        self.is_task = True
        self.priority = priority

    def complete_task(self):
        if self.is_task:
            self.completed_at = datetime.datetime.now()

    def to_dict(self):
        return {
            "title": self.title,
            "content": self.content,
            "created_at": str(self.created_at),
            "updated_at": str(self.updated_at),
            "categories": self.categories,
            "tags": self.tags,
            "is_task": self.is_task,
            "priority": self.priority,
            "completed_at": str(self.completed_at) if self.completed_at else None
        }

    @classmethod
    def from_dict(cls, note_dict):
        note = cls(
            note_dict["title"],
            note_dict["content"],
            [],
            note_dict.get("tags", [])
        )
        note.is_task = note_dict.get("is_task", False)
        note.priority = note_dict.get("priority", None)
        note.completed_at = datetime.datetime.strptime(note_dict["completed_at"], "%Y-%m-%d %H:%M:%S") if note_dict.get("completed_at") else None

        # Retrieve category objects from names
        for category_name in note_dict.get("categories", []):
            category = Category(category_name)
            note.add_category(category)

        return note

    def render_content(self):
        return markdown2.markdown(self.content) if self.content else ""

    def __str__(self):
        categories_str = ", ".join(category.name for category in self.categories)
        task_info = f"Task Priority: {self.priority}, Completed: {self.completed_at}" if self.is_task else "Not a task"
        return f"{self.title}\nCreated: {self.created_at}\nUpdated: {self.updated_at}\nCategories: {categories_str}\n{self.content}\n{task_info}\n"


class Notebook:
    def __init__(self):
        self.notes = []

    def add_note(self, note):
        self.notes.append(note)

    def view_notes(self):
        if not self.notes:
            print("No notes available.")
        else:
            for i, note in enumerate(self.notes, 1):
                print(f"Note {i}:\n{note.render_content()}")

    def save_to_file(self, filename):
        with open(filename, "w") as file:
            notes_data = [note.to_dict() for note in self.notes]
            json.dump(notes_data, file, indent=4)

    def load_from_file(self, filename):
        with open(filename, "r") as file:
            notes_data = json.load(file)
            self.notes = [Note.from_dict(note_dict) for note_dict in notes_data]

# Example Usage:
personal_category = Category("Personal")
work_category = Category("Work")

# Create notes with categories and tags
note1 = Note("Meeting Notes", "Discussed project timeline and deliverables.", [work_category.name], ["meeting", "project"])
note2 = Note("Grocery List", "Milk, eggs, bread, vegetables", [personal_category.name], ["groceries"])
note3 = Note("Task: Complete Report", "Finish the quarterly report", [work_category.name], ["task"])
note4 = Note("Task: Buy Birthday Gift", "Buy a gift for Alice's birthday", [personal_category.name], ["task"])

# Create a notebook
notebook = Notebook()

# Add notes to the notebook
notebook.add_note(note1)
notebook.add_note(note2)
notebook.add_note(note3)
notebook.add_note(note4)

# View all notes in the notebook
print("All Notes:")
notebook.view_notes()

# Mark a note as a task
note3.mark_as_task(priority="High")

# Complete a task
note4.complete_task()

# Save notes to a file
notebook.save_to_file("notes.json")

# Create a new notebook
new_notebook = Notebook()

# Load notes from the file
new_notebook.load_from_file("notes.json")

# View notes in the new notebook
print("\nLoaded Notes:")
new_notebook.view_notes()

# Search for notes containing a keyword
new_notebook.search_notes("project")

# Filter notes by category
new_notebook.filter_by_category("Work")

# Filter notes by tag
new_notebook.filter_by_tag("task")

# Filter notes by date range
new_notebook.filter_by_date_range("2024-01-01", "2024-03-01")


: 